# 1. Menghitung HAZ-Score

## Import Library

In [62]:
import gdown
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

## Data Understanding

In [64]:
dataset = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset.csv')
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,Laki-laki,19,91.6,13.3
1,1,Laki-laki,20,77.7,8.5
2,2,Laki-laki,10,79.0,10.3
3,3,Perempuan,2,50.3,8.3
4,4,Perempuan,5,56.4,10.9


## Data Preparation

### Encoding Data Kategori

In [65]:
label = LabelEncoder()
dataset['Jenis Kelamin'] = label.fit_transform(dataset['Jenis Kelamin'])
dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg)
0,0,0,19,91.6,13.3
1,1,0,20,77.7,8.5
2,2,0,10,79.0,10.3
3,3,1,2,50.3,8.3
4,4,1,5,56.4,10.9


### Hitung HAZ-Score

In [67]:
# # Contoh WHO Height-for-Age Z-score lookup
# Contoh WHO Height-for-Age Z-score lookup
who_haz_lookup = {
    (0, 1): (49.9, 1.9),
    (0, 0): (49.1, 1.8),
    (6, 1): (67.6, 2.6),
    (6, 0): (65.7, 2.5),
    (12, 1): (76.1, 2.9),
    (12, 0): (74.0, 2.8),
    (24, 1): (87.1, 3.1),
    (24, 0): (85.7, 3.0),
    (36, 1): (95.2, 3.5),
    (36, 0): (94.0, 3.4),
    (48, 1): (102.3, 3.8),
    (48, 0): (101.6, 3.7),
    (60, 1): (109.2, 4.2),
    (60, 0): (108.5, 4.2),
}

def calculate_haz(row):
    umur = row["Umur (bulan)"]
    jk = row["Jenis Kelamin"]
    tinggi = row["Tinggi Badan (cm)"]

    # Find the closest age group for the given gender
    closest_key = None
    min_diff = float('inf')
    for (lookup_age, lookup_jk) in who_haz_lookup.keys():
        if lookup_jk == jk:
            diff = abs(lookup_age - umur)
            if diff < min_diff:
                min_diff = diff
                closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age < umur: # Prefer the closest age less than current if tie
                 closest_key = (lookup_age, lookup_jk)
            elif diff == min_diff and lookup_age > umur and closest_key and closest_key[0] > umur: # If closest so far is also > umur, take this one if closer
                 closest_key = (lookup_age, lookup_jk)


    if closest_key in who_haz_lookup:
        median, sd = who_haz_lookup[closest_key]
        # Handle potential division by zero if SD is 0
        if sd != 0:
            return (tinggi - median) / sd
        else:
            return None # Return None if SD is 0 to avoid error
    return None

dataset["HAZ"] = dataset.apply(calculate_haz, axis=1)

# Klasifikasi stunting
def classify_stunting(haz):
    if pd.isna(haz):
        return "Unknown"
    elif haz < -3:
        return "Severely Stunted"
    elif haz < -2:
        return "Stunted"
    else:
        return "Normal"

dataset["Stunting_Status"] = dataset["HAZ"].apply(classify_stunting)
dataset.to_csv("/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset_with_status.csv", index=False)

dataset.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.966667,Normal
1,1,0,20,77.7,8.5,-2.666667,Stunted
2,2,0,10,79.0,10.3,1.785714,Normal
3,3,1,2,50.3,8.3,0.210526,Normal
4,4,1,5,56.4,10.9,-4.307692,Severely Stunted


# 2. Klasifikasi Stunting

## Data Understanding

In [68]:
stunting = pd.read_csv('/content/drive/MyDrive/Capstone-ML/Capstone/stunting_dataset_with_status.csv')

# Display basic info and first few rows
stunting.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.966667,Normal
1,1,0,20,77.7,8.5,-2.666667,Stunted
2,2,0,10,79.0,10.3,1.785714,Normal
3,3,1,2,50.3,8.3,0.210526,Normal
4,4,1,5,56.4,10.9,-4.307692,Severely Stunted


In [69]:
stunting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Id                 100000 non-null  int64  
 1   Jenis Kelamin      100000 non-null  int64  
 2   Umur (bulan)       100000 non-null  int64  
 3   Tinggi Badan (cm)  100000 non-null  float64
 4   Berat Badan (kg)   100000 non-null  float64
 5   HAZ                100000 non-null  float64
 6   Stunting_Status    100000 non-null  object 
dtypes: float64(3), int64(3), object(1)
memory usage: 5.3+ MB


## Encoding Data

In [70]:
label = LabelEncoder()
stunting['Stunting_Status'] = label.fit_transform(stunting['Stunting_Status'])
stunting.head()

,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
0,0,0,19,91.6,13.3,1.966667,0
1,1,0,20,77.7,8.5,-2.666667,2
2,2,0,10,79.0,10.3,1.785714,0
3,3,1,2,50.3,8.3,0.210526,0
4,4,1,5,56.4,10.9,-4.307692,1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Feature *Scaling*

In [72]:
fitur = ['Jenis Kelamin', 'Umur (bulan)', 'Tinggi Badan (cm)', 'Berat Badan (kg)']

scaler = StandardScaler()
stunting[fitur] = scaler.fit_transform(stunting[fitur])

stunting.sample(10)


,Id,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),HAZ,Stunting_Status
83606,83606,1.003586,-1.249033,-0.583820,-1.563049,8.736842,0
31953,31953,1.003586,-0.832345,-0.874293,-0.230024,-1.692308,0
38670,38670,1.003586,-0.971241,-1.182370,-0.502689,-3.038462,1
84516,84516,1.003586,-1.665721,-2.300250,-2.259858,-1.526316,0
97141,97141,1.003586,1.389990,0.525258,0.618264,-2.580645,2
79795,79795,1.003586,1.667782,1.572720,0.133528,1.258065,0
15486,15486,-0.996426,0.556614,1.211830,0.739448,4.607143,0
93122,93122,-0.996426,1.112198,0.789324,1.708921,-1.200000,0
71392,71392,-0.996426,1.528886,0.630884,0.345600,-1.800000,0
83457,83457,-0.996426,-0.832345,-0.522205,0.194120,0.600000,0


## Data Splitting

In [73]:
# 5. Pilih fitur dan target
X = stunting[["Jenis Kelamin", "Umur (bulan)", "Tinggi Badan (cm)", "Berat Badan (kg)"]]
y = stunting["Stunting_Status"]

# 6. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Modelling

In [74]:
# 8. Model klasifikasi
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 9. Evaluasi model
y_pred = model.predict(X_test)
print("Classification Report:\n", classification_report(y_test, y_pred, target_names=label.classes_))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

# 10. Membuat DataFrame hasil prediksi
df_prediksi = X_test.copy()
df_prediksi["Actual_Label"] = label.inverse_transform(y_test)
df_prediksi["Predicted_Label"] = label.inverse_transform(y_pred)

# 11. Tampilkan 10 hasil prediksi pertama
df_prediksi.head(10)

Classification Report:
                   precision    recall  f1-score   support

          Normal       1.00      1.00      1.00     16949
Severely Stunted       1.00      1.00      1.00      1231
         Stunted       1.00      1.00      1.00      1820

        accuracy                           1.00     20000
       macro avg       1.00      1.00      1.00     20000
    weighted avg       1.00      1.00      1.00     20000

Confusion Matrix:
 [[16949     0     0]
 [    0  1231     0]
 [    0     0  1820]]


,Jenis Kelamin,Umur (bulan),Tinggi Badan (cm),Berat Badan (kg),Actual_Label,Predicted_Label
75721,1.003586,-1.526825,-1.631283,-0.684465,Normal,Normal
80184,1.003586,-0.554553,-0.627831,0.890928,Normal,Normal
19864,-0.996426,1.112198,1.484698,0.497080,Normal,Normal
76699,-0.996426,0.139927,0.815731,0.103232,Normal,Normal
92991,-0.996426,-0.137865,-0.266941,1.193889,Normal,Normal
76434,-0.996426,0.556614,0.604478,0.739448,Normal,Normal
84004,1.003586,-0.137865,-0.284545,0.769744,Stunted,Stunted
80917,-0.996426,-0.832345,-0.619029,-0.805649,Normal,Normal
60767,-0.996426,-1.110137,-1.173568,-0.139136,Stunted,Stunted
50074,1.003586,0.834406,0.085148,1.527145,Normal,Normal


In [76]:
# Model TensorFlow
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(4,)),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 kelas: Normal, Stunted, Severely Stunted
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training
model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

# Evaluation
model.evaluate(X_test, y_test)
model.save("/content/drive/MyDrive/Capstone-ML/Capstone/model_stunting.h5")

Epoch 1/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 15s 3ms/step - accuracy: 0.8354 - loss: 0.4442 - val_accuracy: 0.8788 - val_loss: 0.2829
Epoch 2/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 22s 3ms/step - accuracy: 0.8816 - loss: 0.2748 - val_accuracy: 0.8913 - val_loss: 0.2422
Epoch 3/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.8944 - loss: 0.2382 - val_accuracy: 0.9057 - val_loss: 0.2120
Epoch 4/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9104 - loss: 0.2012 - val_accuracy: 0.9178 - val_loss: 0.1864
Epoch 5/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - accuracy: 0.9234 - loss: 0.1773 - val_accuracy: 0.9320 - val_loss: 0.1580
Epoch 6/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9333 - loss: 0.1571 - val_accuracy: 0.9429 - val_loss: 0.1401
Epoch 7/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 12s 3ms/step - accuracy: 0.9423 - loss: 0.1426 - val_accuracy: 0.9473 - val_loss: 0.1281
Epoch 8/20
4500/4500 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.9478 - loss: 0

In [77]:
def predict_stunting(model, scaler, input_data):
    """
    input_data: dict seperti
        {
            "Umur (bulan)": 24,
            "Tinggi Badan (cm)": 85.0,
            "Berat Badan (kg)": 12.5,
            "JK_Code": 1
        }
    """
    df_input = pd.DataFrame([input_data])
    scaled_input = scaler.transform(df_input)
    pred_probs = model.predict(scaled_input)
    pred_class = tf.argmax(pred_probs, axis=1).numpy()[0]
    return label.inverse_transform([pred_class])[0]

In [78]:
save_path = '/content/Model'
tf.saved_model.save(model, save_path)

In [ ]:
!pip install tensorflowjs

In [80]:
!tensorflowjs_converter \
  --input_format=tf_saved_model \
  --output_format=tfjs_graph_model \
  '/content/Model'\
  '/content/drive/MyDrive/Capstone-ML/Capstone/Model-tfjs'

2025-05-22 16:18:14.834183: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747930694.858194   28216 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747930694.865195   28216 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
2025-05-22 16:18:20.054553: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1747930700.190343   28216 devices.cc:67] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
I0000 00:00